In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import re
import os
import boto3
import gensim
from dotenv import load_dotenv
from s3 import create_s3
import cheffrey
load_dotenv()


2023-06-05 19:52:47.432 
  command:

    streamlit run /Users/lando/Projects/cheffrey/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


True

In [3]:
users = {
    'Landon': {'favorites': []},
    'Erin': {'favorites': []},
    'Emily': {'favorites': []},
}
cheffrey.save_yaml('users', users)

In [4]:
cheffrey.load_yaml('users')

{'Emily': {'favorites': []},
 'Erin': {'favorites': []},
 'Landon': {'favorites': []}}

In [2]:
collection_url = 'https://www.bonappetit.com/simple-cooking/quick'

In [6]:
from cheffrey import load_local_recipes
from recipe_scrapers import scrape_me as scrape_recipe
from bs4 import BeautifulSoup
import requests

def pull_existing_recipes():
    master_recipes = load_local_recipes()
    titles = list(master_recipes.keys())
    urls = [r['canonical_url'] for r in master_recipes.values()]
    return titles, urls

In [7]:
base_url = collection_url.split('.com')[0] + '.com/'
res = requests.get(collection_url)
res.raise_for_status()
soup = BeautifulSoup(res.text, features='lxml')

if base_url == 'https://cooking.nytimes.com/':
    recipe_cards = soup.find_all('a', {'class': 'image-anchor card-link'})
    source = 'New York Times'
elif base_url == 'https://www.bonappetit.com/':
    recipe_cards = soup.find_all('a', {'href': re.compile(r'/recipe')})
    source = 'Bon Appetit'
else:
    raise ValueError(f"Unrecognized source: {base_url}")

existing_titles, existing_urls = pull_existing_recipes()

In [13]:
card = recipe_cards[2]

In [15]:
extension = card.get('href')
url = base_url + extension
url_to_download = url

In [18]:
recipe = scrape_recipe(url_to_download)
recipe = recipe.to_json()

In [40]:
recipes = cheffrey.load_local_recipes()

In [43]:
urls = []
new_recipes = {}
for recipe in recipes.values():
    if recipe['canonical_url'] in urls:
        continue
    else:
        urls.append(recipe['canonical_url'])
        recipe['title'] = recipe['title'].replace(' ({source})', '')
        new_recipes[recipe['title']] = recipe


In [45]:
import json
with open(cheffrey.ROOT_DIR/'data/recipes2.json', 'w') as f:
    json.dump(new_recipes, f)

In [41]:
len(set(urls))

1262

In [42]:
len(urls)

1887

In [29]:
x = [c for c in recipes.values() if c['canonical_url'] == 'https://cooking.nytimes.com/recipes/1023492-roasted-mushroom-and-halloumi-grain-bowl']

In [32]:
x[1]['title']

'Roasted Mushroom and Halloumi Grain Bowl ({source})'

In [26]:
import pandas as pd
pd.Series(urls).value_counts()

https://cooking.nytimes.com/recipes/1023492-roasted-mushroom-and-halloumi-grain-bowl        2
https://cooking.nytimes.com/recipes/1017704-vegetarian-skillet-chili                        2
https://cooking.nytimes.com/recipes/1020508-honey-apple-bread-pudding                       2
https://cooking.nytimes.com/recipes/1016380-beet-and-arugula-salad-with-berries             2
https://cooking.nytimes.com/recipes/1023833-whole-wheat-buttermilk-pancakes                 2
                                                                                           ..
https://cooking.nytimes.com/recipes/1023815-miso-butter-mushrooms-with-silky-eggs           1
https://cooking.nytimes.com/recipes/1023926-pan-seared-pork-chops-with-charred-pineapple    1
https://cooking.nytimes.com/recipes/1017082-creme-brulee-french-toast                       1
https://cooking.nytimes.com/recipes/1023369-snapper-escovitch                               1
https://cooking.nytimes.com/recipes/1016035-spicy-hot-chocol

In [9]:
cleaned_string = re.sub(r'[^a-zA-Z]', '', 'howdy123')
cleaned_string

'howdy'

In [62]:
def recipe_embedding(recipe):

    d = {
        'title': .5,
        'instructions': 1,
        'ingredients': 1,
        'category': 3
    }  

    l = []
    for type, modifier in d.items():
        text = recipe[type]
        if type == 'ingredients':
            text = ' '.join(text)
            text = remove_non_alphabetic_chars(text)
        embeddings = cheffrey.get_embedding(model, title)
        if embeddings is not None:
            l.append(embeddings * modifier)

    if len(l) == 0:
        return np.zeros(shape=(model.vector_size))
    
    recipe_embeddings = np.column_stack([l])
    recipe_embeddings = np.mean(recipe_embeddings, axis = 0)

    return recipe_embeddings


# Generate embeddings for each recipe
recipe_embeddings = {}
for title, recipe in recipes.items():
    recipe_embeddings[title] = recipe_embedding(recipe)

In [74]:
from annoy import AnnoyIndex

# Create an AnnoyIndex with the desired embedding dimensions
embedding_dim = model.vector_size
# embedding_dim = len(next(iter(recipe_embeddings.values())))
annoy_index = AnnoyIndex(embedding_dim, metric='euclidean')

# Add the recipe embeddings to the AnnoyIndex
for i, embedding in enumerate(recipe_embeddings.values()):
    annoy_index.add_item(i, embedding)

# Build the index to enable searching
annoy_index.build(n_trees=1000)

annoy_index.save('annoy_index.ann')

True

In [72]:
new_index = AnnoyIndex(model.vector_size, metric = 'euclidean')
new_index.load('annoy_index.ann')

True

In [ ]:
AnnoyIndex.

In [64]:
recip

get_best_recipe('Asian lunch')

['Brookies', 'Gado-Gado', 'Chepa Vepudu (Fish Fry)', 'Manicotti', 'Adasi']

In [38]:
embeddings.shape

(300,)